# FOSSGIS Atlas – Main Notebook
Zentrale Steuerung für das Projekt Konferenzkartografierung 2025+.

In [ ]:
pip install pandas

In [14]:
pip install matplotlib.pyplot


ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
Note: you may need to restart the kernel to use updated packages.


In [13]:
import sys
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path


print("Python:", sys.version)
print("Pandas:", pd.__version__)

ModuleNotFoundError: No module named 'matplotlib'

In [15]:
import requests
import pandas as pd

# Pretalx JSON-Export URL
url = "https://pretalx.com/fossgis2025/schedule/export/schedule.json"
response = requests.get(url)
data = response.json()

# Events extrahieren (verschachtelt: Tag → Raum → Liste von Talks)
events = []
for day in data['schedule']['days']:
    for room_name, talks in day['rooms'].items():
        for talk in talks:
            talk['room'] = room_name  # sicherstellen, dass 'room' immer gesetzt ist
            events.append(talk)

# In DataFrame laden
df = pd.DataFrame(events)

# Basis-Statistiken
print("🔢 Gesamtanzahl der Events:", len(df))
print("\n📦 Alle 'type'-Einträge:", df['type'].dropna().unique())
print("\n🏛️ Alle Räume:", df['room'].dropna().unique())
print("\n🧵 Alle Tracks:", df['track'].dropna().unique())


KeyError: 'days'

In [16]:
import requests
import json

# Pretalx JSON URL
url = "https://pretalx.com/fossgis2025/schedule/export/schedule.json"
response = requests.get(url)

# Optional: Prüfen, ob der Request geklappt hat
if response.status_code == 200:
    data = response.json()
    # Optional schön drucken
    print(json.dumps(data, indent=2, ensure_ascii=False))
else:
    print("Fehler beim Abrufen der JSON:", response.status_code)


{
  "$schema": "https://c3voc.de/schedule/schema.json",
  "generator": {
    "name": "pretalx",
    "version": "2024.4.0.dev0"
  },
  "schedule": {
    "url": "https://pretalx.com/fossgis2025/schedule/",
    "version": "0.25",
    "base_url": "https://pretalx.com",
    "conference": {
      "acronym": "fossgis2025",
      "title": "FOSSGIS-Konferenz 2025",
      "start": "2025-03-26",
      "end": "2025-03-29",
      "daysCount": 4,
      "timeslot_duration": "00:05",
      "time_zone_name": "Europe/Berlin",
      "colors": {
        "primary": "#EE7F00"
      },
      "rooms": [
        {
          "name": "HS1 (Aula)",
          "slug": "3648-hs1-aula",
          "guid": "849b7717-17b1-5880-b4a5-adc2dcbd12d6",
          "description": null,
          "capacity": 330
        },
        {
          "name": "HS2 (S10)",
          "slug": "3649-hs2-s10",
          "guid": "e8909de4-27ae-5480-a037-b4ba01bac420",
          "description": null,
          "capacity": 289
        },
        {

In [17]:
import requests
import os

# Zielpfad definieren
filepath = "data/2025/pretalx.json"

# Ordner anlegen, falls nicht vorhanden
os.makedirs(os.path.dirname(filepath), exist_ok=True)

# JSON von Pretalx laden
url = "https://pretalx.com/fossgis2025/schedule/export/schedule.json"
response = requests.get(url)

# Datei speichern
if response.status_code == 200:
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(response.text)
    print(f"✅ Pretalx JSON gespeichert unter: {filepath}")
else:
    print("❌ Fehler beim Abrufen der Pretalx JSON:", response.status_code)


✅ Pretalx JSON gespeichert unter: data/2025/pretalx.json


In [21]:
import json

with open("data/2025/pretalx.json", "r", encoding="utf-8") as f:
    raw = json.load(f)

# Zeige Top-Level-Keys
print("Top-Level-Keys:", list(raw.keys()))

# Wenn es schedule.json ist, sollte das erscheinen:
# ['schedule']


Top-Level-Keys: ['$schema', 'generator', 'schedule']


In [22]:
import json

# JSON laden
with open("data/2025/pretalx.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Rekursive Funktion für Ascii-Tree
def print_json_tree(obj, indent="", level=0):
    if isinstance(obj, dict):
        for i, key in enumerate(obj):
            is_last = (i == len(obj) - 1)
            prefix = "└── " if is_last else "├── "
            print(indent + prefix + str(key))
            next_indent = indent + ("    " if is_last else "│   ")
            print_json_tree(obj[key], next_indent, level + 1)
    elif isinstance(obj, list):
        print(indent + f"└── [List of {len(obj)} items]")
        if obj and isinstance(obj[0], (dict, list)):
            print_json_tree(obj[0], indent + "    ", level + 1)
    else:
        print(indent + f"└── {type(obj).__name__}")

# Ausgabe starten
print("📁 JSON Tree:")
print_json_tree(data)


📁 JSON Tree:
├── $schema
│   └── str
├── generator
│   ├── name
│   │   └── str
│   └── version
│       └── str
└── schedule
    ├── url
    │   └── str
    ├── version
    │   └── str
    ├── base_url
    │   └── str
    └── conference
        ├── acronym
        │   └── str
        ├── title
        │   └── str
        ├── start
        │   └── str
        ├── end
        │   └── str
        ├── daysCount
        │   └── int
        ├── timeslot_duration
        │   └── str
        ├── time_zone_name
        │   └── str
        ├── colors
        │   └── primary
        │       └── str
        ├── rooms
        │   └── [List of 11 items]
        │       ├── name
        │       │   └── str
        │       ├── slug
        │       │   └── str
        │       ├── guid
        │       │   └── str
        │       ├── description
        │       │   └── NoneType
        │       └── capacity
        │           └── int
        ├── tracks
        │   └── [List of 16 items]
        │       ├

In [23]:
import json
import pandas as pd

# Pretalx JSON laden
with open("data/2025/pretalx.json", "r", encoding="utf-8") as f:
    raw = json.load(f)

# Zugriff auf neue Struktur (Pretalx ≥ 2024.x)
days = raw['schedule']['conference']['days']

# Events extrahieren
events = []
for day in days:
    for room_name, talks in day['rooms'].items():
        for talk in talks:
            talk['room'] = room_name
            events.append(talk)

# In DataFrame
df = pd.DataFrame(events)

# Anzahl Events pro Track (inkl. "Ungetagged" falls track fehlt)
track_counts = df['track'].fillna("– (ohne Track)").value_counts()

# Ausgabe
print("🎯 Anzahl der Events pro Track:")
print(track_counts)


🎯 Anzahl der Events pro Track:
track
– (ohne Track)                                                 63
Praxisberichte                                                 25
Neuigkeiten und Interessantes aus den Open-Source-Projekten    20
Daten, Datenbanken und Datenprozessierung                      14
Kartographie und Visualisierung                                12
Routing und Mobilität                                          11
Grundlagen Open-Source-GIS und OpenStreetMap                   10
Open Data, Datenschutz und Lizenzen                             8
Geodatenmanagement                                              8
Ausbildung, Lehre, Open Science                                 7
OpenStreetMap                                                   6
Offene Standards, z.B. INSPIRE, OGC                             5
Rasterdaten und Fernerkundung                                   5
Raum-zeitliche Analysen und Modelle                             4
25 Jahre FOSSGIS e.V.                  

In [24]:
# Anzahl Events mit und ohne Track
with_track = df[df['track'].notna()]
without_track = df[df['track'].isna()]

print(f"Mit Track: {len(with_track)}")
print(f"Ohne Track: {len(without_track)}")

# Welche Types haben keinen Track?
print("\n📊 Typenverteilung bei Events ohne Track:")
print(without_track['type'].value_counts())


Mit Track: 140
Ohne Track: 63

📊 Typenverteilung bei Events ohne Track:
type
Sonstiges                              19
Lightning-Talk                         11
Vortrag                                 9
Anwendertreffen / BoF                   7
Poster-Session                          7
Workshop (Präsenz)                      7
Expert:innenfragestunde                 2
Vortrag mit Paper im Academic Track     1
Name: count, dtype: int64
